In [1]:
import vertexai

vertexai.init(project="sascha-playground-doit", location="us-central1")

In [2]:
from IPython.core.interactiveshell import InteractiveShell
import IPython.display

InteractiveShell.ast_node_interactivity = "all"

from vertexai.generative_models import (
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Tool,
)
     

In [20]:
repo_url = "https://github.com/GoogleCloudPlatform/microservices-demo"  # @param {type:"string"}

# The location to clone the repo
repo_dir = "./repo"

In [22]:
import os
from pathlib import Path
import shutil

import git
import magika
import requests

m = magika.Magika()


def clone_repo(repo_url, repo_dir):
    """Clone a GitHub repository."""

    if os.path.exists(repo_dir):
        shutil.rmtree(repo_dir)
    os.makedirs(repo_dir)
    git.Repo.clone_from(repo_url, repo_dir)


def extract_code(repo_dir):
    """Create an index, extract content of code/text files."""

    code_index = []
    code_text = ""
    for root, _, files in os.walk(repo_dir):
        for file in files:
            file_path = os.path.join(root, file)
            relative_path = os.path.relpath(file_path, repo_dir)
            code_index.append(relative_path)

            file_type = m.identify_path(Path(file_path))
            if file_type.output.group in ("text", "code"):
                try:
                    with open(file_path) as f:
                        code_text += f"----- File: {relative_path} -----\n"
                        code_text += f.read()
                        code_text += "\n-------------------------\n"
                except Exception:
                    pass

    return code_index, code_text


def get_github_issue(owner: str, repo: str, issue_number: str) -> str:
    headers = {
        "Accept": "application/vnd.github+json",
        "X-GitHub-Api-Version": "2022-11-28",
    }  # Set headers for GitHub API

    # Construct API URL
    url = f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}"

    try:
        response_git = requests.get(url, headers=headers)
        response_git.raise_for_status()  # Check for HTTP errors
    except requests.exceptions.RequestException as error:
        print(f"Error fetching issue: {error}")  # Handle potential errors

    issue_data = response_git.json()
    if issue_data:
        return issue_data["body"]
    return ""
     

In [23]:
clone_repo(repo_url, repo_dir)

code_index, code_text = extract_code(repo_dir)

I0000 00:00:1727455296.984998 8824089 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [24]:
code_index

['LICENSE',
 'cloudbuild.yaml',
 '.editorconfig',
 'README.md',
 '.gitignore',
 '.gitattributes',
 'skaffold.yaml',
 'terraform/output.tf',
 'terraform/main.tf',
 'terraform/terraform.tfvars',
 'terraform/providers.tf',
 'terraform/README.md',
 'terraform/memorystore.tf',
 'terraform/variables.tf',
 'istio-manifests/frontend-gateway.yaml',
 'istio-manifests/allow-egress-googleapis.yaml',
 'istio-manifests/frontend.yaml',
 'protos/demo.proto',
 'protos/grpc/health/v1/health.proto',
 '.deploystack/test',
 '.deploystack/test.yaml',
 '.deploystack/deploystack.yaml',
 '.deploystack/messages/description.txt',
 '.deploystack/messages/success.txt',
 '.deploystack/scripts/preinit.sh',
 'kubernetes-manifests/emailservice.yaml',
 'kubernetes-manifests/checkoutservice.yaml',
 'kubernetes-manifests/recommendationservice.yaml',
 'kubernetes-manifests/kustomization.yaml',
 'kubernetes-manifests/frontend.yaml',
 'kubernetes-manifests/paymentservice.yaml',
 'kubernetes-manifests/README.md',
 'kubernete

In [25]:
print(code_text)

----- File: LICENSE -----

                                 Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or indirect, to cause the
      direction or management of such entity, whether by contract or
      otherwise, or (ii) ownership of fifty percent (50%) or more of the
      outstanding share

In [26]:

MODEL_ID = "gemini-1.5-flash-002"  # @param {type:"string"}

model = GenerativeModel(
    MODEL_ID,
    system_instruction=[
        "You are a coding expert.",
        "Your mission is to answer all code related questions with given context and instructions.",
    ],
)
     

In [27]:
def get_code_prompt(question):
    """Generates a prompt to a code related question."""

    prompt = f"""
    Questions: {question}

    Context:
    - The entire codebase is provided below.
    - Here is an index of all of the files in the codebase:
      \n\n{code_index}\n\n.
    - Then each of the files is concatenated together. You will find all of the code you need:
      \n\n{code_text}\n\n

    Answer:
  """

    return prompt

In [28]:
question = """
  Give me a summary of this codebase, and tell me the top 3 things that I can learn from it.
"""

prompt = get_code_prompt(question)
contents = [prompt]

# Generate text using non-streaming method
response = model.generate_content(contents)

# Print generated text and usage metadata
print(f"\nAnswer:\n{response.text}")
print(f'\nUsage metadata:\n{response.to_dict().get("usage_metadata")}')
print(f"\nFinish reason:\n{response.candidates[0].finish_reason}")
print(f"\nSafety settings:\n{response.candidates[0].safety_ratings}")


Answer:
This codebase implements an online boutique application as a collection of microservices.  Let's break down the key aspects and learning opportunities:

**Codebase Summary:**

The application, "Online Boutique," is a multi-tiered e-commerce system built using microservices.  Each microservice is responsible for a specific function (e.g., product catalog, cart management, payment processing, etc.) and is written in different programming languages (Go, Java, C#, Python, Node.js).  The services communicate with each other using gRPC.

The codebase includes:

* **Microservice implementations:**  Source code for each microservice in their respective languages.
* **Protocol Buffer definitions:**  `proto` files defining the data structures and service interfaces used for gRPC communication.
* **Kubernetes manifests:**  YAML files describing how the microservices are deployed and managed within a Kubernetes cluster.  These are designed to work with `skaffold` for automated builds and 

In [11]:
question = """
  Provide a getting started guide to onboard new developers to the codebase.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

KeyboardInterrupt: 

In [18]:
question = """
  Find the top 3 most severe issues in the codebase.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

Based

 on the provided codebase, here are three of the most severe issues:



1. **Insecure gRPC connections:**  The `AdServiceClient`, `

AdService`, and other services using gRPC frequently utilize `.usePlaintext()` in their channel creation. This completely disables Transport Layer Security (TLS), leaving g

RPC connections vulnerable to eavesdropping and tampering.  This is a critical security flaw that should be addressed immediately.  All production gRPC services should use secure connections

 (TLS).


2. **Unimplemented features and missing error handling:** Several services contain placeholder comments like "TODO(arbrown) Implement OpenTelemetry stats" and "raise Exception('cloud mail client not implemented')" within the `AdService

` and `EmailService` respectively.  These indicate incomplete functionality.  Additionally, many places lack robust error handling, often just logging errors and continuing without proper responses or fallback mechanisms. For example, the `getRecommendations` in `handlers

.go` silently ignores errors and proceeds, masking potential problems.  Incomplete features and inadequate error handling can lead to application instability and unexpected behavior.


3. **Potential for Denial of Service (DoS) in `productcatalogservice`:** The `productcatalogservice` has a dynamic catalog reloading feature that reloads the

 catalog on *every* request. This introduces a significant performance bottleneck and creates a vulnerability to denial-of-service attacks where a large number of requests can overwhelm the service.  The intended functionality should be implemented more efficiently and securely (e.g. using a background process for reloading).


These three issues represent significant

 risks to the security and stability of the application.  Addressing these is crucial before deploying to a production environment.  Other issues exist (commented-out code, inconsistent logging practices, etc.), but these three present the greatest immediate danger.


In [19]:

question = """
  Find the most severe bug in the codebase that you can provide a code fix for.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

The

 most severe bug is in the `src/productcatalogservice/catalog_loader

.go` file. The `loadCatalog` function is called on every request

, which causes significant performance issues and makes the application unresponsive.  This is because it reads and parses the `products.json` file on each request,

 creating unnecessary overhead.

Here's a corrected version of `src/productcatalogservice/catalog_loader.go`:

```go
// Copyright 

...

		return err
	}
	defer rows.Close()

	catalog.Products = catalog.Products[:0]
	for rows.Next() {
		product := &pb.Product{}
		product.PriceUsd = &pb.Money{}

		var categories string
		err =

 rows.Scan(&product.Id, &product.Name, &product.Description,
			&product.Picture, &product.PriceUsd.CurrencyCode, &product.PriceUsd.Units,
			&product.PriceUsd.Nanos, &categories)
		if err != nil

 {
			log.Warnf("failed to scan query result row: %v", err)
			return err
		}
		categories = strings.ToLower(categories)
		product.Categories = strings.Split(categories, ",")

		catalog.Products = append(catalog.Products, product

)
	}

	log.Info("successfully parsed product catalog from AlloyDB")
	return nil
}

```

This revised code loads the product catalog only once, during initialization,  avoiding repeated file I/O or database queries on every request. The `once.Do` function ensures thread

 safety for the initialization.  The `catalog` variable now stores the parsed data, and `parseCatalog()` simply returns this cached data.  This significantly improves the application's performance and responsiveness.


In [20]:
question = """
    Provide a troubleshooting guide to help resolve common issues.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

##

 Online Boutique Troubleshooting Guide

This guide helps resolve common issues encountered while building, deploying

, or running the Online Boutique application.  The issues are categorized for easier navigation

.

**I. Build Issues:**

* **Problem:**  `go build` or `skaffold build` fails with errors related to missing dependencies.


    * **Cause:**  Missing or improperly configured Go modules.
    * **Solution:**
        1. Ensure that Go modules are enabled (`GO

...

VPC, firewall rules) to ensure connectivity.  If using a managed Istio, check the CSM status.
        3. Carefully review the Terraform configuration and scripts for any errors. Use `terraform plan` to see the intended changes.
        4. Try the deployment again after some time; temporary network

 glitches can cause failures.  Ensure that you have a correctly configured billing account linked to the project.

**V. General Troubleshooting:**

* **Check logs:** Regularly check the logs of all services to identify errors or warning messages. Use `kubectl logs <pod-name> -n <namespace>` for Kubernetes pods

 and the appropriate logging mechanisms for other services.
* **Use a debugger:** For debugging code in individual services, utilize a debugger (e.g., `delve` for Go, or your IDE's debugger for other languages).
* **Simplify the setup:** If encountering complex issues, try deploying a simplified

 version of the application (e.g., without certain features or integrations) to isolate the problem.
* **Consult documentation:** Refer to the documentation for each service, Kubernetes, and other relevant tools.
* **Community support:** Search for solutions online or seek help from the community (e.g., Stack Overflow

, GitHub issues).


This troubleshooting guide is not exhaustive, but it covers many of the common problems encountered.  Remember to provide specific error messages and logs when seeking further assistance.


In [21]:

question = """
  How can I make this application more reliable? Consider best practices from https://www.r9y.dev/
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

The

 provided codebase is a large microservices application.  To improve its reliability,

 focusing on several key areas based on the principles outlined in [https://www.

r9y.dev/](https://www.r9y.dev/) is crucial.  R9Y's principles emphasize simplicity, testability

, and observability.  Here's a breakdown of how to apply these to this project:


**1. Simplicity & Maintainability:**

* **

...

 **Immutable Infrastructure:** The use of Terraform is good for infrastructure management.  Maintain an infrastructure as code (IaC) approach to ensure that the environment is consistent and reproducible.  Avoid manual changes to the infrastructure.

* **Automated Deployments:** The CI pipeline shows some automation but could be improved.  Implement

 robust continuous integration and continuous delivery (CI/CD) pipelines to automate the build, test, and deployment process.


**5. Specific Recommendations Based on the Code:**

* **`productcatalogservice`:** The dynamic catalog reloading feature is buggy and inefficient.  Fix this bug or remove the feature.  If

 the need for dynamic updates remains, explore a more robust mechanism (e.g., using a message queue or a database update strategy).

* **`cartservice`:** The choice of database (Redis, Spanner, AlloyDB) is determined by environment variables.  This makes the deployment configuration unnecessarily complicated.  

Consider using Kustomize overlays to switch the database based on environment.

* **`frontend`:** The platform detection logic is fragile.  Enhance the logic to make it more robust.


**6. Security:**

* **Secure Communication:** Ensure all communication between microservices is encrypted (using TLS).

*

 **Authentication & Authorization:** Implement proper authentication and authorization mechanisms to protect your services from unauthorized access.


By addressing these aspects, you'll significantly enhance the reliability, maintainability, and scalability of the Online Boutique application.  Remember to focus on incremental improvements, thoroughly testing each change before deploying it to production.


In [22]:
question = """
  How can you secure the application?
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

The

 application's security can be improved in several ways, based on the provided code

base:

**1. Secure Communication:**

* **gRPC:** The

 application uses gRPC for inter-service communication.  While gRPC is inherently more secure than HTTP, ensure that TLS/SSL is enabled for all gRPC

 connections in production.  The current codebase shows plaintext connections (`usePlaintext()` in `AdServiceClient.java` and `grpc.WithInsecure()`

...

 some OTel versions).  Regularly update dependencies to benefit from improved security and performance.

**6. Infrastructure Security:**

* **Kubernetes:** Leverage Kubernetes security features like Role-Based Access Control (RBAC), Network Policies (partially implemented in `kustomize/components/network-policies`), and Pod

 Security Policies (if supported by your Kubernetes version) to secure your deployments.

* **Cloud Provider Security:**  If deployed on Google Cloud, leverage Google Cloud's security features, including VPC networking, firewall rules, and Identity and Access Management (IAM).  The `release` cluster setup uses some of these,

 but requires more thorough integration and consistency.


**7. Runtime Security:**

* **Seccomp Profiles:** Consider using seccomp profiles to limit the system calls available to containers, enhancing runtime security. The Helm chart's values.yaml has this option.

* **Runtime Application Self Protection (RASP):**

 Integrate a RASP solution to detect and prevent attacks within the running applications.


**Implementation Notes:**

Addressing these security concerns will require significant modifications across the entire application's code and infrastructure.  A phased approach is recommended, prioritizing the most critical vulnerabilities first (secrets management, secure communication).  It's also

 crucial to follow secure coding practices throughout the development process.


In [23]:

question = """
  Create a quiz about the concepts used in my codebase to help me solidify my understanding.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)

Here

's a quiz to help solidify your understanding of the concepts used in the provided

 codebase.  The quiz focuses on key architectural and technological choices, not on

 the minutiae of individual services.

**Instructions:** Answer the following questions to the best of your ability.  The provided codebase should give you all the

 information you need.

**Part 1: Architecture and Deployment**

1. **Microservices Architecture:** What is the overall architecture of the Online Boutique application

...

. **Error Handling:** Examine the error handling mechanisms in one or more of the services. How are errors detected, handled, and propagated?  How are they logged, and what information is included in the logs?

12. **Dependency Management:** Analyze the dependency management approach in either the Go or Python services.

 What tools are used, and how do they ensure version consistency and dependency resolution?

13. **Testing:**  Review the test suites of one or more of the services. What types of tests are included (unit tests, integration tests, etc.)? What testing framework(s) are used?

14

. **Configuration:** Describe the different ways the application's configuration is handled (environment variables, configuration files, etc.). How are sensitive data (such as database passwords) protected?

15. **Code Style and Linting:** Identify the code style guidelines used (`.editorconfig`, linters, etc.) and

 how consistency is enforced during the development process.



This quiz is designed to be open-ended to encourage deeper reflection and understanding.  After completing the quiz, review your answers and the codebase to identify areas where your knowledge might be strengthened.  Focus on *why* certain technologies or patterns were chosen, as well

 as their strengths and limitations.


In [24]:

question = """
  Please write an end-to-end quickstart tutorial that introduces AlloyDB,
  shows how to configure it with the CartService,
  and highlights key capabilities of AlloyDB in context of the Online Boutique application.
"""

prompt = get_code_prompt(question)
contents = [prompt]

responses = model.generate_content(contents, stream=True)
for response in responses:
    IPython.display.Markdown(response.text)
     

This

 tutorial demonstrates how to configure AlloyDB with the CartService in the Online Boutique application

.  It's assumed you have a working Kubernetes cluster and the Online Boutique

 application already deployed (refer to the main README for instructions). This tutorial focuses solely on the AlloyDB integration.


**1. Prerequisites:**

*   A

 Google Cloud project with the AlloyDB API enabled.
*   The Online Boutique application deployed to your Kubernetes cluster.  This includes the `cartservice` deployment

...

_VAL/${PROJECT_ID}/g" components/alloydb/kustomization.yaml
sed -i "s/REGION_VAL/${REGION}/g" components/alloydb/kustomization.yaml

```

**4. Apply Kustomize Changes:**

Apply the updated Kustomize configuration

 to your cluster:


```bash
kubectl apply -k .
```

This will update the `cartservice` deployment to use AlloyDB.  You might need to wait for the pods to restart and become ready.


**5. Verification:**

After deployment, verify the integration:

*   Check the

 `cartservice` logs for successful connection to AlloyDB.
*   Use the `frontend` to add items to the cart and confirm that the data is persisted in AlloyDB.


**Key AlloyDB Capabilities Highlighted:**

*   **Scalability:** AlloyDB offers high scalability, making it suitable for handling

 the increasing demands of the Online Boutique application's shopping cart.
*   **Performance:**  Its optimized architecture ensures fast transaction processing, improving user experience.
*   **Reliability:**  As a fully managed service, AlloyDB reduces operational overhead and ensures high availability.
*   **Security:** AlloyDB provides

 built-in security features to protect sensitive customer data.


This streamlined tutorial showcases AlloyDB integration.  Remember to adjust configurations based on your specific AlloyDB setup and cluster environment.  Refer to the AlloyDB and Kubernetes documentation for further details.


In [25]:
### Fetches commit IDs from a local Git repository on a specified branch.

repo = git.Repo(repo_dir)
branch_name = "main"
commit_ids = [
    commit.hexsha for commit in repo.iter_commits(branch_name)
]  # A list of commit IDs (SHA-1 hashes) in reverse chronological order (newest first)

if len(commit_ids) >= 2:
    diff_text = repo.git.diff(commit_ids[0], commit_ids[1])

    question = """
      Given the above git diff output, Summarize the important changes made.
    """

    prompt = diff_text + question + code_text
    contents = [prompt]

    responses = model.generate_content(contents, stream=True)
    for response in responses:
        IPython.display.Markdown(response.text)

The

 most significant changes are updates to the `langchain` and related packages.  

The `langchain-google-genai` package was downgraded from version 

2.0.0 to 1.0.10, and the `langchain` package was downgraded from 0.3.0 to 

0.2.16.  `langchain-core` was also updated to version 0.2.40.  These changes likely address

 compatibility issues or bugs in the newer versions.  The `requirements.txt` file reflects these dependency updates.
